In [1]:
import FINE as fn
import FINE.utils as utils
import pandas as pd
import numpy as np

import inspect
import json

In [13]:
numberOfTimeSteps = 4
hoursPerTimeStep = 2190

# Create an energy system model instance 
esM = fn.EnergySystemModel(locations={'ElectrolyzerLocation', 'IndustryLocation'}, 
                            commodities={'electricity', 'hydrogen'}, 
                            numberOfTimeSteps=numberOfTimeSteps,
                            commodityUnitsDict={'electricity': r'kW$_{el}$', 'hydrogen': r'kW$_{H_{2},LHV}$'},
                            hoursPerTimeStep=hoursPerTimeStep, costUnit='1 Euro', 
                            lengthUnit='km', 
                            verboseLogLevel=2)

### Buy electricity at the electricity market
costs = pd.DataFrame([np.array([ 0.05, 0., 0.1, 0.051,]),np.array([0., 0., 0., 0.,])],
                        index = ['ElectrolyzerLocation', 'IndustryLocation']).T
revenues = pd.DataFrame([np.array([ 0., 0.01, 0., 0.,]),np.array([0., 0., 0., 0.,])],
                        index = ['ElectrolyzerLocation', 'IndustryLocation']).T
maxpurchase = pd.DataFrame([np.array([1e6, 1e6, 1e6, 1e6,]),np.array([0., 0., 0., 0.,])],
                        index = ['ElectrolyzerLocation', 'IndustryLocation']).T * hoursPerTimeStep
esM.add(fn.Source(esM=esM, name='Electricity market', commodity='electricity', 
                    hasCapacityVariable=False, operationRateMax = maxpurchase,
                    commodityCostTimeSeries = costs,  
                    commodityRevenueTimeSeries = revenues,  
                    )) # eur/kWh

### Electrolyzers
esM.add(fn.Conversion(esM=esM, name='Electrolyzers', physicalUnit=r'kW$_{el}$',
                      commodityConversionFactors={'electricity':-1, 'hydrogen':0.7},
                      hasCapacityVariable=True, 
                      investPerCapacity=500, # euro/kW
                      opexPerCapacity=500*0.025, 
                      interestRate=0.08,
                      economicLifetime=10))

### Hydrogen filled somewhere
esM.add(fn.Storage(esM=esM, name='Pressure tank', commodity='hydrogen',
                   hasCapacityVariable=True, capacityVariableDomain='continuous',
                   stateOfChargeMin=0.33, 
                   investPerCapacity=0.5, # eur/kWh
                   interestRate=0.08,
                   economicLifetime=30))

### Hydrogen pipelines
esM.add(fn.Transmission(esM=esM, name='Pipelines', commodity='hydrogen',
                        hasCapacityVariable=True,
                        investPerCapacity=0.177, 
                        interestRate=0.08, 
                        economicLifetime=40))

### Industry site
demand = pd.DataFrame([np.array([0., 0., 0., 0.,]), np.array([6e3, 6e3, 6e3, 6e3,]),],
                index = ['ElectrolyzerLocation', 'IndustryLocation']).T * hoursPerTimeStep
esM.add(fn.Sink(esM=esM, name='Industry site', commodity='hydrogen', hasCapacityVariable=False,
                operationRateFix = demand,
                ))


In [15]:
demand

,ElectrolyzerLocation,IndustryLocation
0,0.0,13140000.0
1,0.0,13140000.0
2,0.0,13140000.0
3,0.0,13140000.0


In [3]:
# Get all input properties of the esM
inputkwargs = inspect.getfullargspec(fn.EnergySystemModel.__init__)

In [4]:
inputkwargs.args

['self',
 'locations',
 'commodities',
 'commodityUnitsDict',
 'numberOfTimeSteps',
 'hoursPerTimeStep',
 'costUnit',
 'lengthUnit',
 'verboseLogLevel']

In [5]:
esmDict = {}        
# Loop over all props
for arg in inputkwargs.args:
    if not arg is 'self':
        print(getattr(esM,arg))
        print('-----------------------------------------------------------------')
        esmDict[arg] = getattr(esM,arg)

{'ElectrolyzerLocation', 'IndustryLocation'}
-----------------------------------------------------------------
{'hydrogen', 'electricity'}
-----------------------------------------------------------------
{'electricity': 'kW$_{el}$', 'hydrogen': 'kW$_{H_{2},LHV}$'}
-----------------------------------------------------------------
4
-----------------------------------------------------------------
2190
-----------------------------------------------------------------
1 Euro
-----------------------------------------------------------------
km
-----------------------------------------------------------------
2
-----------------------------------------------------------------


In [7]:
esmDict

{'locations': {'ElectrolyzerLocation', 'IndustryLocation'},
 'commodities': {'electricity', 'hydrogen'},
 'commodityUnitsDict': {'electricity': 'kW$_{el}$',
  'hydrogen': 'kW$_{H_{2},LHV}$'},
 'numberOfTimeSteps': 4,
 'hoursPerTimeStep': 2190,
 'costUnit': '1 Euro',
 'lengthUnit': 'km',
 'verboseLogLevel': 2}

In [8]:
compDict = {}

In [9]:
componentModel = esM.componentModelingDict['SourceSinkModel']

In [12]:
getattr(componentModel.componentsDict['Industry site'], 'operationRateFix')

In [ ]:
componentname = 'Electricity market'

In [ ]:
classname = type(componentModel.componentsDict[componentname]).__name__
classname

In [ ]:
compDict[classname]

In [ ]:
if not 'Source' in compDict:
            compDict[classname] = utils.PowerDict()

compDict

In [ ]:
compDict[classname][componentname] = utils.PowerDict()

In [ ]:
compDict

In [ ]:
componentModel.componentsDict

In [ ]:
component = componentModel.componentsDict[componentname]

In [ ]:
component['operationRateMax']

In [ ]:
class_ = getattr(fn, classname)
class_

In [ ]:
inputkwargs = inspect.getfullargspec(class_.__init__)
inputkwargs

In [ ]:
'operationRateMax'

In [ ]:
compDict[classname][componentname]['operationRateMax'] = getattr(component,'operationRateMax')

In [ ]:
getattr(component,'opexIfBuilt')

In [ ]:
compDict

In [ ]:
for prop in inputkwargs.args:
            
    if (prop is not 'self') and (prop is not 'esM'):
        compDict[classname][componentname][prop] = getattr(component,prop)
        print(compDict)

In [ ]:
# Loop over all components
for modelname in esM.componentModelingDict.keys():
    # Get all component models
    componentModel = esM.componentModelingDict[modelname]
    
    # Loop over all components belonging to the model
    for componentname in componentModel.componentsDict:
        print(f'componentname is {componentname}')
        print('===================================================================')       
        

        # Get class name of component
        classname = type(componentModel.componentsDict[componentname]).__name__
        
        if not classname in compDict:
            compDict[classname] = utils.PowerDict()
            
        compDict[classname][componentname] = utils.PowerDict()
        component = componentModel.componentsDict[componentname]
         
        
        # Get class
        class_ = getattr(fn, classname)
        
        
        # Get input arguments of the class
        inputkwargs = inspect.getfullargspec(class_.__init__)

        # Loop over all input props
        for prop in inputkwargs.args:
            
            if (prop is not 'self') and (prop is not 'esM'):
                compDict[classname][componentname][prop] = getattr(component,prop)
                

# TEST OPTIMIZATION

In [ ]:
# 8. Optimize energy system model

esM.cluster(numberOfTypicalPeriods=3)

esM.optimize(timeSeriesAggregation=True, solver = 'gurobi')